**Отношения:**


*   продукты
*   категории
*   связи

Каждому продукту может соответствовать **несколько категорий или ни одной**.

А каждой категории может соответствовать **несколько продуктов или ни одного**.

In [2]:
!pip install findspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=c31e40a06b942d81fcc6344a6f802b22d7fd17dd6f4e861168e4b77f53492585
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


Создаем DataFrame для заданных сущностей

In [7]:
from pyspark.sql import SparkSession, Row

# Создание сессии Spark
spark = SparkSession.builder.appName("product").getOrCreate()

# DataFrame для продуктов
product_data = [
    Row(product_id=1, product_name='Microsoft Office'),
    Row(product_id=2, product_name='Kaspersky Anti-Virus'),
    Row(product_id=3, product_name='Adobe Photoshop'),
    Row(product_id=4, product_name='Windows 10'),
    Row(product_id = 5, product_name='ProCreate')
]
product_df = spark.createDataFrame(product_data)

# DataFrame для категорий
category_data = [
    Row(category_id=1, category_name='OS'),
    Row(category_id=2, category_name='Antivirus Software'),
    Row(category_id=3, category_name='Graphic Editors'),
    Row(category_id=4, category_name='Graphic Design Software')

]
category_df = spark.createDataFrame(category_data)

# DataFrame для связей между продуктами и категориями
product_category_data = [
    Row(product_id=2, category_id=2),  # Kaspersky Anti-Virus: Antivirus Software
    Row(product_id=3, category_id=3),  # Adobe Photoshop: Graphic Editors
    Row(product_id=5, category_id=3),  # ProCreate: Graphic Editors
    Row(product_id=4, category_id=1),  # Windows 10: OS
    Row(product_id=3, category_id=4),  # Adobe Photoshop: Graphic Design Software
    Row(product_id=5, category_id=4)   # ProCreate: Graphic Design Software
]
product_category_df = spark.createDataFrame(product_category_data)

# Вывод DataFrame
product_df.show()
category_df.show()
product_category_df.show()


+----------+--------------------+
|product_id|        product_name|
+----------+--------------------+
|         1|    Microsoft Office|
|         2|Kaspersky Anti-Virus|
|         3|     Adobe Photoshop|
|         4|          Windows 10|
|         5|           ProCreate|
+----------+--------------------+

+-----------+--------------------+
|category_id|       category_name|
+-----------+--------------------+
|          1|                  OS|
|          2|  Antivirus Software|
|          3|     Graphic Editors|
|          4|Graphic Design So...|
+-----------+--------------------+

+----------+-----------+
|product_id|category_id|
+----------+-----------+
|         2|          2|
|         3|          3|
|         5|          3|
|         4|          1|
|         3|          4|
|         5|          4|
+----------+-----------+



Функция для выполнения запроса:

метод на PySpark, который в одном датафрейме вернет все пары «Имя продукта – Имя категории» и имена всех продуктов, у которых нет категорий.

In [8]:
def product_category(product_df, product_category_df, category_df):
  product_category_pairs = product_df.join(product_category_df, 'product_id', how='left') \
    .join(category_df, 'category_id', how='left') \
    .select('product_name', 'category_name')
  return product_category_pairs

In [9]:
product_category_pairs = product_category(product_df, product_category_df, category_df)
product_category_pairs.show()

+--------------------+--------------------+
|        product_name|       category_name|
+--------------------+--------------------+
|    Microsoft Office|                NULL|
|          Windows 10|                  OS|
|           ProCreate|     Graphic Editors|
|     Adobe Photoshop|     Graphic Editors|
|Kaspersky Anti-Virus|  Antivirus Software|
|           ProCreate|Graphic Design So...|
|     Adobe Photoshop|Graphic Design So...|
+--------------------+--------------------+



In [10]:
spark.stop()